# Machine Learning

*Yuriy Sverchkov*

**BMI 773 Clinical Research Informatics**

March 23, 2020

## Lecture Goals

* Be able to define Machine Learning
* List the most common tasks that machine learning is used to solve
* Be able to distinguish between a supervised and unsupervised task
* Describe how some widely-used machine learning methods work
* Describe how machine learning methods are trained and evaluated

## What is Machine Learning

## Machine Learning Tasks
* Classification
* Regression
* Clustering

## The ML workflow for supervised classification/regression

1. Have a question about data of the form "can some set of measurements/observations predict an outcome?"
2. Turn your measurements and observations into numbers (some are numbers already, categorical variables should usually be 1-hot)
3. Turn your outome into a number (this is where it becomes clear if we're doing classification or regression)
4. Fit a mathematical model on some known outcomes
5. Test if it works
6. You now have a way of predicting the outcome from your measurements

## Logistic Regression

Most of you are probably familiar with logistic regression in one context or another.

### Examples
(todo)

### Mathematical notation

The model that is learned in logistic regression has the form:

$y = logit( \beta x )$

Note the simple structure of the model: features are multiplied by weights and summed, then, a function is applied to transform the result.

Let's look at the logit function more closely:

In [2]:
# Plot logit here

This function has an unbounded domain (goes all the way fron $-\infty$ to $+\infty$) and a range from 0 to 1.

Note that it is also *monotonically increasing*, that is, if $a > b$ then $logit(a) > logit(b)$.

This property makes it particularly easy to interpret what the weights mean: higher positive weights mean that x correlates with y, weights near 0 mean that x does not affect y much, and hight negative weights 